# 1. Elevation Data Reading

In this notebook we access and compare elevation data for four different data sets: SRTM3, SRTM30, EARTH2014 and GMTED2010. 

In [9]:
import numpy as np
import os
import pickle 
import re
import rasterio
from osgeo import gdal
import sys

Change the following paths to point to the correct locations of the datasets. Note that the datasets were too large for my linux personal laptop, so they were stored in an external drive. 

In [10]:
SRTM3_PATH = '/media/sushi/Expansion/Data/SRTM3/'
SRTM30_PATH = '/media/sushi/Expansion/Data/SRTM30/'
GMTED2010_PATH = '/media/sushi/Expansion/Data/GMTED2010_MEAN_HEIGHTS/'
EARTH2014_PATH = '/media/sushi/Expansion/Data/EARTH2014/'

## 1.1 SRTM3 (3 arcseconds)

For SRTM3, the provided files range from $-56^\deg$ to $60^\deg$ in latitude and from $-180^\deg$ to $180^\deg$ in longitude. For reference the file "N14E100.hgt" has N14E100 at its bottom left corner when viewed as a 1201 by 1201 sample file, and the top left corner has coordinate N15E100.

In [11]:
SRTM3_SAMPLES = 1201

# default signature for all functions is lat followed by lon

def read_elevation_from_file_SRTM3(filename, latitude, longitude):
    dataset = gdal.Open(SRTM3_PATH + filename)
    elevation_data = dataset.ReadAsArray()
        
    lat_name, lon_name = re.findall(r"\d+", filename)
    lat_file, lon_file = int(lat_name), int(lon_name)
    
    directions = re.findall(r"[A-Za-z]", filename)
    if directions[0] == 'S':
        lat_file *= -1
    if directions[1] == 'W':
        lon_file *= -1
        
    lat_row = SRTM3_SAMPLES - 1 - int(round((latitude - lat_file) * (SRTM3_SAMPLES - 1)))
    lon_row = int(round((longitude - lon_file) * (SRTM3_SAMPLES - 1)))
    return elevation_data[lat_row, lon_row]

In [14]:
def get_elevation_SRTM3(latitude, longitude):
    lat_name = int(np.floor(latitude))
    lon_name = int(np.floor(longitude))
    
    filename = ""
    if lat_name >= 0:
        filename += ("N" + '%02d' % lat_name)
    else:
        filename += ("S" + '%02d' % abs(lat_name))
    
    if lon_name >= 0:
        filename += ("E" + '%03d' % lon_name)
    else:
        filename += ("W" + '%03d' % abs(lon_name))
    
    filename += ".hgt"
    if os.path.exists(SRTM3_PATH + filename):
        return read_elevation_from_file_SRTM3(filename, latitude, longitude)
    else:
        # ask what to do since SRTM3 only covers S56 to N60
        print("!")
        return 0

## 1.2 SRTM30 (30 arcseconds)

In [12]:
SRTM30_SAMPLES = 120

def earth_elevation_grid_SRTM30(data_path):
    longitudes = ['N90', 'N40', 'S10']
    latitudes = ['W180', 'W140', 'W100', 'W060', 'W020', 'E020', 'E060', 'E100', 'E140']
    
    bands_to_vertically_stack = []
    for lon in longitudes:
        files_to_horizontally_stack = []
        for lat in latitudes:
            filename = lat + lon + '.DEM'
            with rasterio.open(SRTM30_PATH + filename) as data:
                elevation_data = data.read(1)
            files_to_horizontally_stack.append(elevation_data)
        bands_to_vertically_stack.append(np.hstack(files_to_horizontally_stack))
    return np.vstack(bands_to_vertically_stack)

EARTH_ELEVATION_GRID_SRTM30 = earth_elevation_grid_SRTM30(SRTM30_PATH)

In [13]:
def get_elevation_SRTM30(latitude, longitude): 
    # SRTM30 gives -60 --> 90 long and full lat. 
    assert -60 < latitude <= 90
    row = int(np.round((90 - latitude) * SRTM30_SAMPLES))
    col = int(np.round((longitude + 180) * SRTM30_SAMPLES))
    return EARTH_ELEVATION_GRID_SRTM30[row, col]

## 1.3 EARTH2014 (60 arcseconds)

In [10]:
EARTH2014_SAMPLES = 60

def earth_elevation_grid_EARTH2014(data_path):
    with rasterio.open(data_path + 'Earth2014.SUR2014.1min.geod.grd') as elevation_data:
        return elevation_data.read(1)

EARTH_ELEVATION_GRID_EARTH2014 = earth_elevation_grid_EARTH2014(EARTH2014_PATH)

In [11]:
def get_elevation_EARTH2014(latitude, longitude):
    row = int(np.floor((90 - latitude) * EARTH2014_SAMPLES))
    col = int(np.floor((longitude + 180) * EARTH2014_SAMPLES))
    return EARTH_ELEVATION_GRID_EARTH2014[row, col]

## 1.4 GMTED2010 (30 arcseconds)
Code to collect data from USGS Global Multi-resolution Terrain Elevation Data (GMTED2010) at 30 arc-second resolution. Data set taken from USGS Earth Explorer website (https://earthexplorer.usgs.gov/). 

In [ ]:
GMTED2010_SAMPLES = 120

def generate_earth_elevation_grid_GMTED2010(path):
    longitudes = ['S90', 'S70', 'S50', 'S30', 'S10', 'N10', 'N30', 'N50', 'N70']
    latitudes = ['W180', 'W150', 'W120', 'W090', 'W060', 'W030', 'E000', 'E030', 'E060', 'E090', 'E120', 'E150']
    elevation_grid = None

    for longitude in longitudes:
        horizontal_band = None
        for latitude in latitudes:
            filename = longitude.lower()[1:] + longitude.lower()[0] + \
                latitude.lower()[1:] + latitude.lower()[0] + "_20101117_gmted_mea300.tif"
            file = gdal.Open(path + filename)
            band = file.GetRasterBand(1)

            # Read the band data as a NumPy array
            band_data = band.ReadAsArray()
            # print(band_data)
            if horizontal_band is None: 
                horizontal_band = band_data
            else:
                horizontal_band = np.concatenate((horizontal_band, band_data), axis = 1)
            # NOTE that 10n030e is at the lower left hand corner of the 
            # file when viewed as image. Goes up to 30n30e
            
        if elevation_grid is None: 
            elevation_grid = horizontal_band
        else: 
            elevation_grid = np.concatenate((horizontal_band, elevation_grid))
    return elevation_grid

ELEVATION_GRID_GMTED2010 = generate_earth_elevation_grid_GMTED2010(GMTED2010_PATH)

In [ ]:
def get_elevation_GMTED2010(latitude, longitude):
    row = int(np.round(((90 - latitude) * GMTED2010_SAMPLES)) % (180 * GMTED2010_SAMPLES))
    col = int(np.round(((longitude + 180) * GMTED2010_SAMPLES)) % (360 * GMTED2010_SAMPLES))
    return ELEVATION_GRID_GMTED2010[row][col]

## Benchmarking

In [15]:
# These are coordinates for Catania Italy, Agerola Italy, Albox Spain, Alcantarilha Portugal, 
# MaunaKea CFHT, Nederland, CO, US

# We expect values of 7, 660, 485, 65, ...

coordinates = [(37.5079, 15.0830), (40.6239444, 14.5640556), (37.4055556, -2.1519444), (37.133, -8.365), 
               (19.8252806, -155.4688694), (39.9872083, -105.4455694), (37.0176306, -122.0795), 
               (34.49125, 132.6511111)]

for coordinate in coordinates: 
    lat, lon = coordinate
    print(get_elevation_SRTM3(lat, lon))
    print(get_elevation_SRTM30(lat, lon))
    #print(get_elevation_EARTH2014(lat, lon))
    #print(get_elevation_GMTED2010(lat, lon))
    print("________")

48
24
________
656
403
________
490
470
________
63
39
________
4178
3969
________
2502
2536
________
341
329
________
225
258
________
